In [ ]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFileApply
from cognite.client.exceptions import CogniteException

from tests.auth_utils import get_toolkit_client

In [ ]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

In [ ]:
SPACE = "sp_upgrade"

In [ ]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

# Cascading Delete

In [6]:
file = FileMetadataWrite(
    external_id="ts_case2_cascading_delete",
    name="Case 2 Cascading Delete",
)

In [ ]:
# Ensure the file does not exist before running the test
client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)

In [7]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
is_string,False
metadata,{}
is_step,False
security_categories,[]
id,6626764436391143
created_time,2025-06-03 05:42:01.039000
last_updated_time,2025-06-03 05:42:01.039000


In [8]:
filecontent = b"This is filecontent"

In [9]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

In [10]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    updated

,value
external_id,ts_case2_cascading_delete
name,Case 2 Cascading Delete
is_string,False
metadata,{}
is_step,False
security_categories,[]
id,6626764436391143
created_time,2025-06-03 05:42:01.039000
last_updated_time,2025-06-03 05:42:10.503000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [11]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name=file.name,
)

In [12]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case2_cascading_delete,1,True,2025-06-03 05:42:44.245,2025-06-03 05:42:44.245


In [13]:
deleted = client.data_modeling.instances.delete(cognite_file.as_id()).nodes
deleted

[NodeId(space='sp_upgrade', external_id='ts_case2_cascading_delete')]

In [14]:
client.time_series.retrieve(external_id=file.external_id)